In [2]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import json
import os
import re
import pandas as pd
from src import util
from sklearn.preprocessing import MultiLabelBinarizer

In [3]:
with open('config/data-params.json') as f:
    params = json.load(f)
sitemap = params['sitemap']
out = params['path']
num = params['num']
cat = params['cat']

In [4]:
def all_smali(mypath):
    sub_dir = [os.path.join(mypath, o) for o in os.listdir(mypath) 
                    if os.path.isdir(os.path.join(mypath,o))]
    smali_by_app = []
    for i in sub_dir:
        smali_by_app += [util.app_to_smali(i)]
    return smali_by_app

In [5]:
all_smali = all_smali(out + '/' + cat[0])

In [6]:
smali_df = pd.DataFrame(all_smali)

## A

In [7]:
def extract_api(x):
    return set(re.findall('invoke-\w+ {.*}, (.*?)\\(', '\n'.join(x)))

In [8]:
apis_by_app = smali_df.apply(extract_api, axis = 1)

In [9]:
mlb_A = MultiLabelBinarizer(sparse_output = True)
A = mlb_A.fit_transform(apis_by_app)

In [10]:
apis = mlb_A.classes_

In [11]:
A.shape

(21, 199879)

In [12]:
def A(smali_df):
    apis_by_app = smali_df.apply(extract_api, axis = 1)
    mlb_A = MultiLabelBinarizer(sparse_output = True)
    A = mlb_A.fit_transform(apis_by_app)
    apis = mlb_A.classes_
    return A, apis

## B

In [13]:
def extract_block(x):
    return list(set(re.findall('\.method([\S\s]*?)\.end method', '\n'.join(x))))

In [14]:
code_blocks = smali_df.apply(extract_block, axis = 1)

In [15]:
block_df = code_blocks.explode().reset_index().drop('index', axis = 1)

In [16]:
B_dic= {}

In [17]:
def extract_api_from_blocks(block):
    apis = set(re.findall('invoke-\w+ {.*}, (.*?)\\(',block))
    for api in apis:
        if not api in B_dic.keys():
            B_dic[api] = apis
        else:
            B_dic[api] = B_dic[api].union(apis)

In [19]:
block_df[0].dropna().apply(extract_api_from_blocks)

0         None
1         None
2         None
3         None
4         None
          ... 
593791    None
593792    None
593793    None
593794    None
593795    None
Name: 0, Length: 593793, dtype: object

In [20]:
mlb_b = MultiLabelBinarizer(sparse_output = True)
B = mlb_b.fit_transform(B_dic.values())
B.shape

(199879, 199879)

In [31]:
def B(smali_df):
    code_blocks = smali_df.apply(extract_block, axis = 1)
    block_df = code_blocks.explode().reset_index().drop('index', axis = 1)
    B_dic= {}
    
    def extract_api_from_blocks(block):
        apis = set(re.findall('invoke-\w+ {.*}, (.*?)\\(',block))
        for api in apis:
            if not api in B_dic.keys():
                B_dic[api] = apis
            else:
                B_dic[api] = B_dic[api].union(apis)
    
    block_df[0].dropna().apply(extract_api_from_blocks)
    mlb_b = MultiLabelBinarizer(sparse_output = True)
    B = mlb_b.fit_transform(B_dic.values())
    return B

## P

In [22]:
def extract_package(x):
    return re.search('(\[*[ZBSCFIJD]|\[*L[\w\/$-]+;)->', x)[1] 

In [23]:
P_dic = {}

In [24]:
api_df = pd.DataFrame({'apis': apis})

In [25]:
api_df['package'] = api_df['apis'].apply(extract_package)

In [26]:
api_by_pac = api_df.groupby('package').apis.apply(set).to_frame().reset_index().apis#.to_dict().values()

In [27]:
def extract_api_package(x):
    for api in x:
        if not api in P_dic.keys():
            P_dic[api] = x
        else:
            P_dic[api] = P_dic[api].union(x)

In [28]:
api_by_pac.apply(extract_api_package)

0        None
1        None
2        None
3        None
4        None
         ... 
47281    None
47282    None
47283    None
47284    None
47285    None
Name: apis, Length: 47286, dtype: object

In [29]:
mlb_p = MultiLabelBinarizer(sparse_output = True)
P = mlb_p.fit_transform(P_dic.values())
P.shape

(199879, 199879)

In [30]:
def P(apis):
    P_dic = {}
    api_df = pd.DataFrame({'apis': apis})
    api_df['package'] = api_df['apis'].apply(extract_package)
    api_by_pac = api_df.groupby('package').apis.apply(set).to_frame().reset_index().apis
    def extract_api_package(x):
        for api in x:
            if not api in P_dic.keys():
                P_dic[api] = x
            else:
                P_dic[api] = P_dic[api].union(x)
    api_by_pac.apply(extract_api_package)
    mlb_p = MultiLabelBinarizer(sparse_output = True)
    P = mlb_p.fit_transform(P_dic.values())
    return P